<a href="https://colab.research.google.com/github/mayahar/Harel-et-elle/blob/main/Analysis_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Background code - download data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import warnings
warnings.filterwarnings("ignore")

from IPython.display import clear_output
clear_output(wait=False)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import seaborn as sns
from scipy.stats import norm
glob.glob('/content/gdrive/My Drive/Thesis*')

['/content/gdrive/My Drive']

In [ ]:
!apt-get install r-base
!pip install -q rpy2
packnames = ('lme4', 'lmerTest', 'emmeans', "geepack")
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import StrVector
utils = importr("utils")
utils.chooseCRANmirror(ind=1)
utils.install_packages(StrVector(packnames))

%load_ext rpy2.ipython
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Thesis/Analysis/Pilot exploratory stage Metrics updated.tsv', sep='\t')

In [ ]:
df.loc[df['Number_of_Events.SD_start'] > 1, 'Number_of_Events.SD_start'] = 1
df['Exp_bin'] = df['Group'].apply(lambda x: 0 if 'cadet' in str(x).lower() else 1)
df['Role_bin'] = df['Group'].apply(lambda x: 0 if 'nav' in str(x).lower() else 1)
TOI_list = ['BHA','Coriolis','GLS','Leans']
filtered_df = df[df['TOI'].isin(TOI_list)].copy()
for col in filtered_df.columns:
    if filtered_df[col].dtype == 'object':
        filtered_df.loc[:, col] = filtered_df[col].astype(str)

# Demographic Effects

## How does experience effect

In [ ]:
%%R -i df -i filtered_df -o result
library(lme4)
library(lmerTest)

filtered_df$Aircraft <- as.factor(filtered_df$Aircraft)
model <- glmer(Number_of_Events.SD_start ~ experience + (1 | Recording),
                data = filtered_df, family = binomial)
model_summary <- summary(model)
result <- as.data.frame(model_summary$coefficients)

Loading required package: Matrix

Attaching package: ‘lmerTest’

The following object is masked from ‘package:lme4’:

    lmer

The following object is masked from ‘package:stats’:

    step

boundary (singular) fit: see help('isSingular')


In [ ]:
z = norm.ppf(0.975)
result["OR"] = np.exp(np.clip(result["Estimate"], -700, 700))
result["CI Lower"] = np.exp(np.clip(result["Estimate"] - z * result["Std. Error"], -700, 700))
result["CI Upper"] = np.exp(np.clip(result["Estimate"] + z * result["Std. Error"], -700, 700))
result = pd.DataFrame(result)
result

,Estimate,Std. Error,z value,Pr(>|z|),OR,CI Lower,CI Upper
(Intercept),0.258348,0.259495,0.995579,0.319455,1.294789,0.778604,2.153184
experience,-0.001018,0.015843,-0.064232,0.948786,0.998983,0.968439,1.030490


## How does role effect

In [ ]:
%%R -i df -i filtered_df -o result
library(lme4)
library(lmerTest)

filtered_df$Aircraft <- as.factor(filtered_df$Aircraft)
model <- glmer(Number_of_Events.SD_start ~ Role_bin + (1 | Recording),
                data = filtered_df, family = binomial)
model_summary <- summary(model)
result <- as.data.frame(model_summary$coefficients)

boundary (singular) fit: see help('isSingular')


In [ ]:
z = norm.ppf(0.975)
result["OR"] = np.exp(np.clip(result["Estimate"], -700, 700))
result["CI Lower"] = np.exp(np.clip(result["Estimate"] - z * result["Std. Error"], -700, 700))
result["CI Upper"] = np.exp(np.clip(result["Estimate"] + z * result["Std. Error"], -700, 700))
result = pd.DataFrame(result)
result

,Estimate,Std. Error,z value,Pr(>|z|),OR,CI Lower,CI Upper
(Intercept),0.579818,0.236039,2.456455,0.014032,1.785714,1.124334,2.836145
Role_bin,-0.561800,0.302908,-1.854689,0.063641,0.570182,0.314904,1.032402


## Interaction between role and experience


In [ ]:
%%R -i df -i filtered_df -o result
library(lme4)
library(lmerTest)

filtered_df$Aircraft <- as.factor(filtered_df$Aircraft)
model <- glmer(Number_of_Events.SD_start ~ Role_bin + Exp_bin + Role_bin*Exp_bin + (1 | Recording),
                data = filtered_df, family = binomial)
model_summary <- summary(model)
result <- as.data.frame(model_summary$coefficients)

boundary (singular) fit: see help('isSingular')


In [ ]:
z = norm.ppf(0.975)
result["OR"] = np.exp(np.clip(result["Estimate"], -700, 700))
result["CI Lower"] = np.exp(np.clip(result["Estimate"] - z * result["Std. Error"], -700, 700))
result["CI Upper"] = np.exp(np.clip(result["Estimate"] + z * result["Std. Error"], -700, 700))
result = pd.DataFrame(result)
result

,Estimate,Std. Error,z value,Pr(>|z|),OR,CI Lower,CI Upper
(Intercept),4.957497e-16,0.447215,1.108526e-15,1.000000,1.000000,0.416226,2.402538
Role_bin,9.531018e-02,0.543559,1.753446e-01,0.860809,1.100000,0.379065,3.192061
Exp_bin,7.985077e-01,0.529677,1.507537e+00,0.131673,2.222222,0.786911,6.275517
Role_bin:Exp_bin,-9.228054e-01,0.658784,-1.400771e+00,0.161283,0.397403,0.109263,1.445402


# Examining eye parameters

In [ ]:
ves = ['Leans', 'Coriolis']
vis = ['GLS','BHA']
HUD_aoi = ['HUD','Center']
ins_aoi = ['Ball','Altitude','Direction','Pitch']
Parameters = ['Number_of_Visits','Number_of_fixations','Number_of_saccades_in_AOI',
              'Total_duration_of_Visit','Total_duration_of_fixations']

produced_columns = []
for parameter in Parameters:
    summed_column_name = f"{parameter}_ins"
    matching_columns = [f"{parameter}.{aoi}" for aoi in ins_aoi if f"{parameter}.{aoi}" in filtered_df.columns]
    if matching_columns:
        filtered_df[summed_column_name] = filtered_df[matching_columns].sum(axis=1)
        produced_columns.append(summed_column_name)  # Add the column name to the list

for parameter in Parameters:
    summed_column_name = f"{parameter}_HUD"
    matching_columns = [f"{parameter}.{aoi}" for aoi in HUD_aoi if f"{parameter}.{aoi}" in filtered_df.columns]
    if matching_columns:
        filtered_df[summed_column_name] = filtered_df[matching_columns].sum(axis=1)
        produced_columns.append(summed_column_name)  # Add the column name to the list




## All parameters on Vestibular illusions

In [ ]:
results_ves = []
for parameter in produced_columns:
    filtered_df = filtered_df[filtered_df['TOI'].isin(ves)].copy()

    if filtered_df.empty:
        continue  # Skip if no rows match the TOI

    for col in filtered_df.columns:
        if filtered_df[col].dtype == 'object':
            filtered_df.loc[:, col] = filtered_df[col].astype(str)

    r.assign("filtered_df", filtered_df)

    # Run logistic regression in R
    r(f'''
    library(lme4)
    library(lmerTest)

    # Convert Aircraft to a factor (categorical variable)
    filtered_df$Aircraft <- as.factor(filtered_df$Aircraft)

    model <- glmer(Number_of_Events.SD_start ~ {parameter} * Role_bin + (1|Recording),
                    data = filtered_df, family = binomial)
    model_summary <- summary(model)
    result <- as.data.frame(model_summary$coefficients)
    ''')

    result = r('result')
    result_df = pandas2ri.rpy2py(result)

    z = norm.ppf(0.975)
    result_df["OR"] = np.exp(np.clip(result_df["Estimate"], -700, 700))
    result_df["CI Lower"] = np.exp(np.clip(result_df["Estimate"] - z * result_df["Std. Error"], -700, 700))
    result_df["CI Upper"] = np.exp(np.clip(result_df["Estimate"] + z * result_df["Std. Error"], -700, 700))

    row_data = {'Predictor': parameter}
    if len(result_df) > 1:  # Ensure there is a second row
            for idx, row in result_df.iterrows():
                if idx in parameter:
                    row_data['parameter.estimate'] = row['Estimate']
                    row_data['parameter.Pr'] = row['Pr(>|z|)']
                    row_data['parameter.OR'] = row['OR']
                    row_data['parameter.CI Lower'] = row['CI Lower']
                    row_data['parameter.CI Upper'] = row['CI Upper']
                else:  # Use 'row name.estimate' for other rows
                    row_data[f'{row.name}.estimate'] = row['Estimate']
                    row_data[f'{row.name}.Pr'] = row['Pr(>|z|)']
                    row_data[f'{row.name}.OR'] = row['OR']
                    row_data[f'{row.name}.CI Lower'] = row['CI Lower']
                    row_data[f'{row.name}.CI Upper'] = row['CI Upper']
    results_ves.append(row_data)

results_df = pd.DataFrame(results_ves)
results_df

## All parameters on Visual illusions

In [ ]:
results_vis = []
for parameter in produced_columns:
    filtered_df = filtered_df[filtered_df['TOI'].isin(vis)].copy()

    if filtered_df.empty:
        continue
    # Skip if no rows match the TOI
    for col in filtered_df.columns:
        if filtered_df[col].dtype == 'object':
            filtered_df.loc[:, col] = filtered_df[col].astype(str)

    r.assign("filtered_df", filtered_df)

    # Run logistic regression in R
    r(f'''
    library(lme4)
    library(lmerTest)

    # Convert Aircraft to a factor (categorical variable)
    filtered_df$Aircraft <- as.factor(filtered_df$Aircraft)

    model <- glmer(Number_of_Events.SD_start ~ {parameter} * Role_bin + (1|Recording),
                    data = filtered_df, family = binomial)
    model_summary <- summary(model)
    result <- as.data.frame(model_summary$coefficients)
    ''')

    result = r('result')
    result_df = pandas2ri.rpy2py(result)

    z = norm.ppf(0.975)
    result_df["OR"] = np.exp(np.clip(result_df["Estimate"], -700, 700))
    result_df["CI Lower"] = np.exp(np.clip(result_df["Estimate"] - z * result_df["Std. Error"], -700, 700))
    result_df["CI Upper"] = np.exp(np.clip(result_df["Estimate"] + z * result_df["Std. Error"], -700, 700))

    row_data = {'Predictor': parameter}
    if len(result_df) > 1:  # Ensure there is a second row
            for idx, row in result_df.iterrows():
                if idx in parameter:
                    row_data['parameter.estimate'] = row['Estimate']
                    row_data['parameter.Pr'] = row['Pr(>|z|)']
                    row_data['parameter.OR'] = row['OR']
                    row_data['parameter.CI Lower'] = row['CI Lower']
                    row_data['parameter.CI Upper'] = row['CI Upper']
                else:  # Use 'row name.estimate' for other rows
                    row_data[f'{row.name}.estimate'] = row['Estimate']
                    row_data[f'{row.name}.Pr'] = row['Pr(>|z|)']
                    row_data[f'{row.name}.OR'] = row['OR']
                    row_data[f'{row.name}.CI Lower'] = row['CI Lower']
                    row_data[f'{row.name}.CI Upper'] = row['CI Upper']
    results_vis.append(row_data)
results_df = pd.DataFrame(results_vis)
results_df

## Triple regression

In [ ]:
# Run logistic regression for triple interaction
triple_interaction_results = []
filtered_df = filtered_df[filtered_df['TOI'].isin(all_toi)].copy()
for param in Parameters:

    value_vars = [f'{param}_ins', f'{param}_HUD']  # Correct column names
    if not all(var in filtered_df.columns for var in value_vars):
        continue  # Skip if the required columns are not present

    df_long = filtered_df.melt(
        id_vars=['Recording', 'TOI', 'Number_of_Events.SD_start', 'Role_bin'],  # Keep Recording and TOI as identifiers
        value_vars=value_vars,
        var_name='ins_hud',  # Column to indicate ins/hud
        value_name='parameter_value'  # Column for parameter values
    )

    # Map 'ins_hud' to binary values: 0 for HUD, 1 for INS
    df_long['ins_hud'] = df_long['ins_hud'].map({f'{param}_ins': 1, f'{param}_HUD': 0})

    # Map TOI to binary values: 0 for vis, 1 for ves
    df_long['TOI_binary'] = df_long['TOI'].apply(lambda x: 0 if x in vis else 1)

    r.assign("df_long", df_long)

    # Run logistic regression in R
    r(f'''
    library(lme4)
    library(lmerTest)

    model <- glmer(Number_of_Events.SD_start ~ parameter_value * TOI_binary * ins_hud + (1|Recording),
                    data = df_long, family = binomial)
    model_summary <- summary(model)
    result <- as.data.frame(model_summary$coefficients)
    ''')

    result = r('result')
    result_df = pandas2ri.rpy2py(result)

    z = norm.ppf(0.975)
    result_df["OR"] = np.exp(np.clip(result_df["Estimate"], -700, 700))
    result_df["CI Lower"] = np.exp(np.clip(result_df["Estimate"] - z * result_df["Std. Error"], -700, 700))
    result_df["CI Upper"] = np.exp(np.clip(result_df["Estimate"] + z * result_df["Std. Error"], -700, 700))

    row_data = {'Predictor': param}
    if len(result_df) > 1:  # Ensure there is a second row
        for idx, row in result_df.iterrows():
            row_data[f'{row.name}.estimate'] = row['Estimate']
            row_data[f'{row.name}.Pr'] = row['Pr(>|z|)']
            row_data[f'{row.name}.OR'] = row['OR']
            row_data[f'{row.name}.CI Lower'] = row['CI Lower']
            row_data[f'{row.name}.CI Upper'] = row['CI Upper']
    triple_interaction_results.append(row_data)

triple_results_df = pd.DataFrame(triple_interaction_results)
triple_results_df